In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import math
import sys
import time
rows = 'ABCDEFGHI'
cols = '123456789'

In [21]:
def cross(a,b):
    return [i+j for i in a for j in b]
boxes=cross(rows,cols)
ru=[cross(r,cols) for r in rows]
cu=[cross(rows,c) for c in cols]
su=[cross(rs,cs) for rs in ('ABC','DEF','GHI') for cs in('123','456','789')]
ul=ru+cu+su
units = dict((s, [u for u in ul if s in u])  for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s]))for s in boxes)

In [22]:
def grid_values(grid):
    assert len(grid) == 81, "Input grid must be a string length of 81 (9x9)"
    boxes = cross(rows,cols)
    return dict(zip(boxes,grid))

In [23]:
def display(values):
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center (width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    return

In [24]:
def grid_values_improved(grid):
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
                values.append(c)
    assert len(values) == 81
    boxes = cross(rows,cols)
    return dict(zip(boxes,values))    

In [25]:
def elimination(values):
    solved_values = [box for box in values.keys() if len(values[box])==1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

In [26]:
def only_choice(values):
    for unit in ul:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values    

In [27]:
def reduce_puzzle(values):
    stalled =False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box])==1])
        elimination(values)
        only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box])==1])
        stalled = solved_values_after == solved_values_before
        if len([box for box in values.keys() if len(values[box])==1])==0:
            return False
    return values    

In [28]:
def search(values):
    values_reduced = reduce_puzzle(values)
    if not values_reduced:
        return False
    else:
        values=values_reduced
    if len([box for box in values.keys() if len(values[box])==1])==81:
        return values   
    possibility_count_list = [(len(values[box]),box) for box in values.keys() if len(values[box])>1]    
    possibility_count_list.sort()
    for(_,t_box_min) in possibility_count_list:
        for i_digit in values[t_box_min]:
            new_values = values.copy()
            new_values[t_box_min]=i_digit
            new_values = search(new_values)
            if new_values:
                return new_values           
    return values

In [29]:
def search(values):
    values_reduced = reduce_puzzle(values)
    if not values_reduced:
        return False
    else:
        values=values_reduced
    if len([box for box in values.keys() if len(values[box])==1])==81:
        return values  
    possibility_count_list = [(len(values[box]),box) for box in values.keys() if len(values[box])>1]
    possibility_count_list.sort()
    for(_,t_box_min) in possibility_count_list:
        for i_digit in values[t_box_min]:
            new_values = values.copy()
            new_values[t_box_min]=i_digit
            new_values = search(new_values)
            if new_values:
                return new_values
            
    return values

In [37]:
p='...8.1.........43.5............7.8........1...2..3....6......75..34........2..6..'

In [38]:
start_time = time.time()
display(grid_values(p))

. . . |8 . 1 |. . . 
. . . |. . . |4 3 . 
5 . . |. . . |. . . 
------+------+------
. . . |. 7 . |8 . . 
. . . |. . . |1 . . 
. 2 . |. 3 . |. . . 
------+------+------
6 . . |. . . |. 7 5 
. . 3 |4 . . |. . . 
. . . |2 . . |6 . . 


In [39]:
p1=grid_values_improved(p)
display(p1)

123456789 123456789 123456789 |    8     123456789     1     |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |    4         3     123456789 
    5     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |123456789     7     123456789 |    8     123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |    1     123456789 123456789 
123456789     2     123456789 |123456789     3     123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
    6     123456789 123456789 |123456789 123456789 123456789 |123456789     7         5     
123456789 123456789     3     |    4     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |    2     123456789 123456789 |    6   

In [40]:
result = search(p1)
display(result)

2 3 7 |8 4 1 |5 6 9 
8 9 6 |7 2 5 |4 3 1 
5 1 4 |3 9 6 |2 8 7 
------+------+------
3 6 1 |5 7 9 |8 4 2 
4 7 5 |6 8 2 |1 9 3 
9 2 8 |1 3 4 |7 5 6 
------+------+------
6 4 2 |9 1 8 |3 7 5 
1 5 3 |4 6 7 |9 2 8 
7 8 9 |2 5 3 |6 1 4 


In [41]:
time_taken=time.time() - start_time
print("\n\n{0} seconds".format(time_taken))



11.268808841705322 seconds
